# Exercise 2

In [47]:
import torch
import torch.cuda
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from torch.utils.tensorboard import SummaryWriter

## Transfer Learning from ImageNet

### Preprocess

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {'cuda' if torch.cuda.is_available() else 'cpu'}")

In [48]:
# Load and normalizde the data

transform = transforms.Compose(
    [transforms.Resize(64),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batchSize = 100
validSize = 0.2 # use 20% of train set as validation

trainValidSet = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testSet = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainSet, validSet = torch.utils.data.random_split(trainValidSet, [int(len(trainValidSet)*(1-validSize)), int(len(trainValidSet)*validSize)])

trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize, shuffle=True)
validLoader = torch.utils.data.DataLoader(validSet, batch_size=batchSize, shuffle=True)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=batchSize, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
Using cuda


In [49]:
# Writer for tensorBoard
writer = SummaryWriter()

### Define train and test functions

In [50]:
def trainNetwork(network, optimizer, lossFunction, trainLoader, validLoader, epochs, device, writer, name):
    network.train()
    for epoch in range(epochs):
        
        ### TRAINING ###
        trainLoss = 0
        correctTrain = 0
        totalTrain = 0
        for batch_nr, (images, labels) in enumerate(trainLoader):
            # Move data to GPU (if exists)
            images, labels = images.to(device), labels.to(device)  

            # Predict
            predictions = network(images)

            # Get loss and backpropogate
            loss = lossFunction(predictions, labels)
            loss.backward() 

            # Optimize parameters (weights and biases) and remove gradients after
            optimizer.step() 
            optimizer.zero_grad()

            # Save loss for whole epoch
            trainLoss += loss.item()
            
            # Calculate training accuracy
            _, predictions = torch.max(predictions, 1) 
            correctTrain += (predictions == labels).sum().item() 
            totalTrain += len(images)

            #print(f'Epoch [{epoch+1}/{epochs}] Batch [{batch_nr}/{len(trainLoader)}]')
        

        trainLoss /= len(trainLoader)
        trainAccuracy = 100 * correctTrain / totalTrain
        writer.add_scalar(f"Loss/train - {name}", trainLoss, epoch)
        writer.add_scalar(f"Accuracy/train - {name}", trainAccuracy, epoch)

        ### VALIDATION ###
        validLoss = 0
        correctValid = 0
        totalValid = 0
        for batch_nr, (images, labels) in enumerate(validLoader):
            # Move data to GPU (if exists)
            images, labels = images.to(device), labels.to(device)  

            # Predict
            predictions = network(images)

            # Get loss
            loss = lossFunction(predictions, labels)

            # Save loss for whole epoch
            validLoss += loss.item()

            # Calculate vaildation accuracy
            _, predictions = torch.max(predictions, 1) 
            correctValid += (predictions == labels).sum().item() 
            totalValid += len(images)

            #print(f'Epoch [{epoch+1}/{epochs}] Batch [{batch_nr}/{len(validLoader)}]')

        validLoss /= len(validLoader)
        validAccuracy = 100 * correctValid / totalValid
        writer.add_scalar(f"Loss/valid - {name}", validLoss, epoch)
        writer.add_scalar(f"Accuracy/valid - {name}", validAccuracy, epoch)

        # Print reuslt of epoch
        print(f'Epoch [{epoch+1}/{epochs}] \t Training Loss: {trainLoss} \t Validation Loss: {validLoss} \t Traning Acc: {trainAccuracy}% \t Validation Acc: {validAccuracy}%')

    writer.flush()

In [51]:
def testNetwork(network, testLoader, device):
    network.eval()
    correctTest = 0
    totalTest = 0

    ### TESTING ###
    with torch.no_grad(): 
        for batch_nr, (images, labels) in enumerate(testLoader):
            # Move data to GPU (if exists)
            images, labels = images.to(device), labels.to(device)  
            
            # Get predictions and get the amount of correct predicitons
            predictions = network(images)
            _, predictions = torch.max(predictions, 1) 
            correctTest += (predictions == labels).sum().item() 
            totalTest += len(images)
            
            #print(f'Batch [{batch_nr}/{len(testLoader)}]', end='\r')
        
    print(f"Test Accuracy: {100 * correctTest / totalTest}%")

### Define our AlexNet

In [52]:
class myAlexNet(nn.Module):
    def __init__(self, pretrained):
        super(myAlexNet,self).__init__()
        self.alexNet = torchvision.models.alexnet(pretrained=pretrained)
        self.out = nn.Linear(1000,10)
    
    def forward(self, x):
        x = F.relu(self.alexNet.forward(x))
        # x = self.alexNet.forward(x)
        return self.out(x)

### Train the network without pretrained weights

In [53]:
learningRate = 0.01

In [54]:
epochs = 20
networkNotTrained = myAlexNet(pretrained=False).to(device)

optimizer = torch.optim.SGD(networkNotTrained.parameters(), lr=learningRate)
lossFunction = nn.CrossEntropyLoss().to(device)

In [55]:
# networkNotTrained.eval()

In [56]:
trainNetwork(networkNotTrained, optimizer, lossFunction, trainLoader, validLoader, epochs, device, writer, "Not pretrained")

Epoch [1/20] 	 Training Loss: 2.3026793217658996 	 Validation Loss: 2.3028131341934204 	 Traning Acc: 10.08% 	 Validation Acc: 9.68%
Epoch [2/20] 	 Training Loss: 2.3025830233097078 	 Validation Loss: 2.3027525472640993 	 Traning Acc: 10.085% 	 Validation Acc: 9.67%
Epoch [3/20] 	 Training Loss: 2.3025386959314345 	 Validation Loss: 2.3026693534851073 	 Traning Acc: 10.095% 	 Validation Acc: 9.76%
Epoch [4/20] 	 Training Loss: 2.3024898266792295 	 Validation Loss: 2.3026162791252136 	 Traning Acc: 10.1175% 	 Validation Acc: 10.08%
Epoch [5/20] 	 Training Loss: 2.3024277901649475 	 Validation Loss: 2.3025527930259706 	 Traning Acc: 10.7725% 	 Validation Acc: 10.18%
Epoch [6/20] 	 Training Loss: 2.3023615777492523 	 Validation Loss: 2.3024608635902406 	 Traning Acc: 10.9925% 	 Validation Acc: 9.67%
Epoch [7/20] 	 Training Loss: 2.3022643733024597 	 Validation Loss: 2.302339415550232 	 Traning Acc: 10.775% 	 Validation Acc: 9.8%
Epoch [8/20] 	 Training Loss: 2.3021331268548964 	 Validatio

In [57]:
testNetwork(networkNotTrained, testLoader, device)

Test Accuracy: 25.13%


### Finetune the network with pretrained weights

In [62]:
epochs = 10
networkPretrained = myAlexNet(pretrained=True).to(device)

for param in networkPretrained.alexNet.parameters():
    param.requires_grad=False
 
optimizer = torch.optim.SGD(networkPretrained.parameters(), lr=learningRate)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [63]:
trainNetwork(networkPretrained, optimizer, lossFunction, trainLoader, validLoader, epochs, device, writer, "Pretrained weights")

Epoch [1/10] 	 Training Loss: 3.0843573772907256 	 Validation Loss: 2.2896007752418517 	 Traning Acc: 41.16% 	 Validation Acc: 44.44%
Epoch [2/10] 	 Training Loss: 2.5662552347779273 	 Validation Loss: 2.8435751891136167 	 Traning Acc: 46.575% 	 Validation Acc: 45.74%
Epoch [3/10] 	 Training Loss: 2.4675160971283914 	 Validation Loss: 3.181151692867279 	 Traning Acc: 48.1475% 	 Validation Acc: 45.25%
Epoch [4/10] 	 Training Loss: 2.3435420536994935 	 Validation Loss: 2.990860027074814 	 Traning Acc: 48.605% 	 Validation Acc: 44.33%
Epoch [5/10] 	 Training Loss: 2.3538567152619363 	 Validation Loss: 3.086433701515198 	 Traning Acc: 49.6425% 	 Validation Acc: 42.17%
Epoch [6/10] 	 Training Loss: 2.298509730696678 	 Validation Loss: 2.0730548346042634 	 Traning Acc: 49.735% 	 Validation Acc: 50.95%
Epoch [7/10] 	 Training Loss: 2.30906494140625 	 Validation Loss: 2.8714713084697725 	 Traning Acc: 49.69% 	 Validation Acc: 48.52%
Epoch [8/10] 	 Training Loss: 2.28601520717144 	 Validation L

In [64]:
testNetwork(networkPretrained, testLoader, device)

Test Accuracy: 57.5%


## Transfer Learning from MNIST

In [65]:
# Load and normalizde the data
transform2 = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batchSize2 = 100
validSize2 = 0.2 # use 20% of train set as validation

trainValidSet2 = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform2)
testSet2 = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform2)

trainSet2, validSet2 = torch.utils.data.random_split(trainValidSet2, [int(len(trainValidSet2)*(1-validSize2)), int(len(trainValidSet2)*validSize2)])

trainLoader2 = torch.utils.data.DataLoader(trainSet2, batch_size=batchSize2, shuffle=True)
validLoader2 = torch.utils.data.DataLoader(validSet2, batch_size=batchSize2, shuffle=True)
testLoader2 = torch.utils.data.DataLoader(testSet2, batch_size=batchSize2, shuffle=False)


In [66]:
# New writer
writer2 = SummaryWriter()

In [ ]:
# Define the network class
class ConvNet(nn.Module):
    def __init__(self, activation):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 100)
        self.fc2 = nn.Linear(100, 10)
        self.activation = activation

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
epochs = 20
networkMnist= ConvNet(F.leaky_relu).to(device)

optimizer = torch.optim.SGD(networkMnist.parameters(), lr=learningRate)

In [ ]:
trainNetwork(networkMnist, optimizer, lossFunction, trainLoader2, validLoader2, epochs, device, writer2, "Train on MNIST")